In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import osmium

import pandas as pd
import numpy as np

from collections import defaultdict

In [2]:
class RelationHandler(osmium.SimpleHandler):

    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.history_relation = []
        self.count_bid = 1
        self.to_bid = defaultdict(lambda: 0)
        
    def relation(self, r):
        tags = dict(r.tags)
        ways_list = []

        if not self.to_bid[r.id]:
            self.to_bid[r.id] = self.count_bid
            self.count_bid +=1

        for member in r.members:
            ways_list.append(member.ref)
            if not self.to_bid[member.ref]:
                self.to_bid[member.ref] = self.to_bid[r.id]

        self.history_relation.append([
                                r.id,
                                self.to_bid[r.id],
                                ways_list,
                                r.version,
                                r.visible,
                                pd.Timestamp(r.timestamp),
                                r.uid,
                                r.changeset,
                                len(r.tags),
                                tags,
                                'R'])

class WayHandler(osmium.SimpleHandler):

    def __init__(self, to_bid):
        osmium.SimpleHandler.__init__(self)
        self.history_way = []
        self.to_bid = to_bid
        max_key = max(self.to_bid, key=self.to_bid.get)
        self.count_bid = self.to_bid[max_key]
        
    def way(self, w):
        tags = dict(w.tags)
        nodes = []

        for i in list(w.nodes):
            nodes.append(i.ref)

        if not self.to_bid[w.id]:
            self.to_bid[w.id] = self.count_bid
            self.count_bid +=1
        
        self.history_way.append([
                                w.id,
                                self.to_bid[w.id],
                                nodes,
                                w.version,
                                w.visible,
                                pd.Timestamp(w.timestamp),
                                w.uid,
                                w.changeset,
                                len(w.tags),
                                tags,
                                'W'])
                                # id, 

In [3]:
h_r = RelationHandler()
h_r.apply_file("data/osm/historical/rec_historical.osm.pbf")
to_bid = h_r.to_bid

h_w = WayHandler(to_bid)
h_w.apply_file("data/osm/historical/rec_historical.osm.pbf")

In [4]:
colnames = ['id', 'bid', 'nodes/ways', 'version', 'visible', 'ts', 'uid', 'chgset', 'ntags', 'tags', 'type']
history = pd.DataFrame(h_r.history_relation + h_w.history_way, columns=colnames)
history = history.sort_values(by=['id', 'ts'])

In [5]:
# number of tag edits, number of users, number of versions, number of tags

def building_qualifier(tags):
   return ( ('building' in tags) or ('building:part' in tags) or (tags.get('type') == 'building') ) and ( (tags.get('location') != 'underground') or ('bridge' not in tags) )


def extract_direct_indicators(df):
   direct_indicators = []

   mp = defaultdict(int)
   for item in df.to_numpy():
      item_id = item[0]
      bid = item[1]
      tags = item[9]
      item_type = item[10]
      qualifies = building_qualifier(tags)

      if(not mp[item_id] and qualifies):
         mp[item_id] = 1

         all_versions = df[df['id']==item_id]
         n = len(all_versions)
         
         num_rollbacks = 0
         num_edits = 0
         num_users = all_versions['uid'].nunique() # Number of unique users contributing to per building
         num_versions = all_versions.iloc[n-1]['version'] # Number of versions of the building
         num_tags = all_versions.iloc[n-1]['ntags'] # Number of tags in the latest version of the building
         visibility = "B" # visibility: D: deleted / NB: not building / B: building

         deletion = 0
         addition = 0
         change = 0
         direct_confirmations = 0

         for i in range(1, n):
            
            prev = all_versions.iloc[i-1] 
            cur = all_versions.iloc[i]

            prev_tags = prev['tags']
            cur_tags = cur['tags']

            prev_n_w = prev['nodes/ways']
            cur_n_w = cur['nodes/ways']

            prev_uid = prev['uid']
            cur_uid = cur['uid']

            # Rollback: 
            # If the geometry or the tags of cur does not match with the prev, check if it matches with any other previous versions. If so, count it as a rollback.
            if(cur_n_w != prev_n_w or cur_tags != prev_tags):
               for k in range(i-1):
                  if(cur_n_w == all_versions.iloc[k]['nodes/ways'] and cur_tags == all_versions.iloc[k]['tags']):
                     num_rollbacks += 1

            # Direct Confirmations:         
            if(cur_uid != prev_uid and cur_n_w == prev_n_w): # If different users mapped the same geometry for a building, we would count it as direct confirmation
               direct_confirmations += 1

            for key in prev_tags: # Number of deleted tags in the newer version
               if key not in cur_tags:
                  deletion += 1

            for key in cur_tags: # Number of newly added tags in the newer version
               if key not in prev_tags:
                  addition += 1

            for key in cur_tags: # Number of tags that were changed in the newer version
               if key in prev_tags:
                  if(cur_tags[key] != prev_tags[key]):
                     change += 1

            if(i == n-1):
               if(not cur['visible']): visibility = "D" # Deleted: if on the latest version -> no tags and no nodes
               elif(not building_qualifier(cur_tags)): visibility = "NB" # Not Building: else if on the latest version -> no tags that qualify for a building

         # Edits
         num_edits = deletion + addition + change

         direct_indicators.append([item_id,
                                 bid,
                                 num_versions,
                                 num_users,
                                 num_edits,
                                 num_tags,
                                 direct_confirmations,
                                 num_rollbacks,
                                 visibility,
                                 item_type])


   colnames = ['id', 'bid', 'nversions', 'nusers', 'nedits', 'ntags', 'dir_confirmations', 'nrollbacks', 'visibility', 'type']
   dir_ind = pd.DataFrame(direct_indicators, columns=colnames)
   
   return dir_ind

In [7]:
dir_ind = extract_direct_indicators(history)
dir_ind

In [10]:
stats = dir_ind.describe()[['nversions', 'nusers', 'nedits', 'ntags', 'dir_confirmations', 'nrollbacks']]
stats.filter(items = ['mean', '50%'], axis=0)


,nversions,nusers,nedits,ntags,dir_confirmations,nrollbacks
mean,1.200821,1.138874,0.461976,1.458597,0.09321,0.001085
50%,1.000000,1.000000,0.000000,1.000000,0.00000,0.000000


In [ ]:
# Have to take a city that has mix of reliable and unreliable data! From that we can acquire a good central tendency value and compare for other regions!!

In [1]:
from util import sidewalk_qualifer
from extract_direct_indicators import *

In [2]:
dir_ind = extract_direct_indicators(sidewalk_qualifer, 'rec')
dir_ind

,id,nversions,nusers,nedits,ntags,dir_confirmations,nrollbacks,visibility,type
0,379240246,2,2,1,2,1,0,B,W
1,379240248,5,5,1,2,1,0,B,W
2,434546645,3,3,5,4,1,0,NB,W
3,469292020,6,3,1,1,1,0,NB,W
4,482849683,4,3,1,3,1,0,B,W
...,...,...,...,...,...,...,...,...,...
480,1035042936,1,1,0,2,0,0,B,W
481,1035042938,1,1,0,2,0,0,B,W
482,1035042941,1,1,0,2,0,0,B,W
483,1035042942,1,1,0,2,0,0,B,W


In [3]:
stats = get_stats(dir_ind)
stats

,nversions,nusers,nedits,ntags,dir_confirmations,nrollbacks
mean,2.103093,1.282474,1.338144,1.371134,0.059794,0.006186
50%,2.000000,1.000000,2.000000,2.000000,0.000000,0.000000
